# 6. Interação

_“Um gráfico não é ‘desenhado’ de uma vez por todas; ele é ‘construído’ e reconstruído até revelar todas as relações constituídas pela interação dos dados. As melhores operações gráficas são aquelas realizadas pelo próprio tomador de decisões.”_ &mdash; [Jacques Bertin](https://books.google.com/books?id=csqX_xnm4tcC)

A visualização oferece um meio poderoso para compreender dados. No entanto, uma única imagem geralmente responde, no máximo, a algumas poucas perguntas. Por meio da _interação_, podemos transformar imagens estáticas em ferramentas de exploração: destacando pontos de interesse, ampliando para revelar padrões mais detalhados e estabelecendo conexões entre múltiplas visualizações para compreender relações multidimensionais.

No centro da interação está a noção de _selection_ (seleção): um meio de indicar ao computador quais elementos ou regiões nos interessam. Por exemplo, podemos passar o mouse sobre um ponto, clicar em múltiplas marcas ou desenhar uma caixa delimitadora ao redor de uma região para destacar subconjuntos dos dados para inspecionar melhor.

Além das codificações visuais e transformações de dados, Altair fornece uma abstração de _selection_ para a criação de interações. Essas seleções abrangem três aspectos:

1. Tratamento de eventos de entrada para selecionar pontos ou regiões de interesse, como passar o mouse, clicar, arrastar, rolar e eventos de toque.
2. Generalização a partir da entrada para formar uma regra de seleção (ou [_predicado_](https://en.wikipedia.org/wiki/Predicate_%28mathematical_logic%29)) que determina se um determinado registro de dados faz parte da seleção.
3. Utilização do predicado de seleção para configurar dinamicamente uma visualização, controlando _conditional encodings_ (codificações condicionais), _filter transforms_ (transformações de filtro) ou _scale domains_ (domínios de escala).


Este notebook introduz seleções interativas e explora como usá-las para criar uma variedade de técnicas de interação, como consultas dinâmicas, arrastamento &amp; zoom, detalhes sob demanda e seleção interativa &amp; _linking_.

_Este notebook é parte do [currículo de visualização de dados](https://github.com/uwdata/visualization-curriculum)._

In [1]:
import pandas as pd
import altair as alt

## 6.1 Datasets

Visualizaremos uma variedade de datasets da coleção [vega-datasets](https://github.com/vega/vega-datasets):

- O dataset `cars` dos anos 1970 e início dos anos 1980,
- O dataset `movies`, usado anteriormente no notebook [Data Transformation](https://github.com/uwdata/visualization-curriculum/blob/master/altair_data_transformation.ipynb),
- O dataset contendo dez anos de preço de ações do [S&amp;P 500](https://en.wikipedia.org/wiki/S%26P_500_Index) (`sp500`) ,
- O dataset `stocks` de uma empresa de tecnologia, e
- O dataset `flights`, incluindo hora de partida, distância e atraso de chegada.

In [2]:
cars = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/cars.json'
movies = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/movies.json'
sp500 = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/sp500.csv'
stocks = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/stocks.csv'
flights = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/flights-5k.json'

## 6.2 Apresentando Seleções

Vamos começar com uma seleção básica: basta clicar em um ponto para destacá-lo. Usando o dataset `cars`, começaremos com um gráfico de dispersão de potência em cavalos versus milhas por galão, com uma codificação de cores para o número de cilíndros no motor do carro.

Além disso, criaremos uma instância de seleção chamando `alt.selection_single()`, indicando que queremos uma seleção definida sobre um _único valor_. Por padrão, a seleção usa um clique do _mouse_ para determinar o valor selecionado. Para registrar uma seleção com um gráfico, precisamos adicioná-la usando o método `.add_selection()`.

Uma vez que nossa seleção foi definida, podemos usá-la como um parametro para _codificações condicionais_, que aplicam uma codificação diferente dependendo se um registro de dados está dentro ou fora da seleção. Por exemplo considere o seguinte código:

~~~ python
color=alt.condition(selection, 'Cylinders:O', alt.value('grey'))
~~~

Esta definição de codificação afirma que os pontos de dados contidos dentro do `selection` devem ser coloridos de acordo com o campo `Cylinder`, enqaunto os pontos de dados não selecionados devem usar um padrão `grey`. Uma seleção vazia inclui todos os pontos de dados e, portanto, incialmente todos os pontos serão coloridos.

Tente clicar em pontos diferentes no gráfico abaixo. O que acontece? (Clique no fundo para limpar o estado de seleção e retornar a uma seleção "vazia".)

In [3]:
selection = alt.selection_single();
  
alt.Chart(cars).mark_circle().add_selection(
    selection
).encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(selection, 'Cylinders:O', alt.value('grey')),
    opacity=alt.condition(selection, alt.value(0.8), alt.value(0.1))
)

alt.Chart(...)

É claro que destacar pontos de dados individualmente, um por vez, não é particularmente empolgante! No entanto, como veremos, seleções de valor único fornecem um bloco de construção útil para interações mais poderosas. Além disso, seleções de valor único são apenas um dos três tipos de seleção fornecidos pelo Altair:

- `selection_single` – seleciona um único valor discreto, por padrão em eventos de clique.
- `selection_multi` – seleciona múltiplos valores discretos. O primeiro valor é selecionado com um clique do _mouse_ e valores adicionais são alternados usando shift-clique.
- `selection_interval` – seleciona um intervalo contínuo de valores, iniciado por um arrasto do _mouse_.

Vamos comparar cada um desses tipos de seleção lado a lado. Para manter nosso código organizado, primeiro definiremos uma função (`plot`) que gera uma especificação de gráfico de dispersão como a acima. Podemos passar uma seleção para a função `plot` para que ela seja aplicada ao gráfico:

In [4]:
def plot(selection):
    return alt.Chart(cars).mark_circle().add_selection(
        selection
    ).encode(
        x='Horsepower:Q',
        y='Miles_per_Gallon:Q',
        color=alt.condition(selection, 'Cylinders:O', alt.value('grey')),
        opacity=alt.condition(selection, alt.value(0.8), alt.value(0.1))
    ).properties(
        width=240,
        height=180
    )

Vamos usar nossa função `plot` para criar três variantes de gráfico, uma por tipo de seleção.

O primeiro gráfico (`single`) replica nosso exemplo anterior. O segundo gráfico (`multi`) permite interações com shift-click para alternar a inclusão de vários pontos na seleção. O terceiro gráfico (`interval`) gera uma região de seleção (ou pincel) ao arrastar o _mouse_. Depois de criada, você pode arrastar a região para selecionar diferentes pontos ou usar a rolagem do _mouse_ quando o cursor estiver dentro da região para ajustar (zoom) seu tamanho.

Tente interagir com cada um dos gráficos abaixo!

In [5]:
alt.hconcat(
  plot(alt.selection_single()).properties(title='Single (Click)'),
  plot(alt.selection_multi()).properties(title='Multi (Shift-Click)'),
  plot(alt.selection_interval()).properties(title='Interval (Drag)')
)

alt.HConcatChart(...)

Os exemplos acima usam interações padrão (clique, shift-clique, arrastar) para cada tipo de seleção. Podemos personalizar ainda mais as interações fornecendo especificações de eventos de entrada usando a [sintaxe do seletor de eventos do Vega](https://vega.github.io/vega/docs/event-streams/). Por exemplo, podemos modificar nossos gráficos `single` e `multi` para serem acionados por eventos de `mouseover` em vez de eventos de `click`.

Segure a tecla shift no segundo gráfico para "pintar" com os dados!

In [6]:
alt.hconcat(
  plot(alt.selection_single(on='mouseover')).properties(title='Single (Mouseover)'),
  plot(alt.selection_multi(on='mouseover')).properties(title='Multi (Shift-Mouseover)')
)

alt.HConcatChart(...)

Agora que abordamos os conceitos básicos das seleções do Altair, vamos fazer um tour pelas várias técnicas de interação que elas permitem!

## 6.3. Consultas Dinâmicas

_Consultas dinâmicas_ permitem a exploração rápida e reversível de dados para isolar padrões de interesse. Conforme definido por [Ahlberg, Williamson, &amp; Shneiderman](https://www.cs.umd.edu/~ben/papers/Ahlberg1992Dynamic.pdf) uma consulta dinâmica:

- Representa uma consulta graficamente,
- Fornece limites visíveis no intervalo de consulta,
- Fornece uma representação gráfica dos dados e do resultado da consulta,
- Fornece feedback imediato do resultado após cada ajuste da consulta,
- Permite que usuários novatos comecem a trabalhar com pouco treinamento.

Uma abordagem comum é manipular parâmetros de consulta usando _widgets_ de interface de usuário padrão, como controles deslizantes, botões de opção e menus suspensos. Para gerar widgets de consulta dinâmica, podemos aplicar uma operação `bind` (vincular) de seleção a um ou mais campos de dados que desejamos consultar.

Vamos construir um gráfico de dispersão interativo que usa uma consulta dinâmica para filtrar a exibição. Dado um gráfico de dispersão de classificações de filmes (do _Rotten Tomates_ e IMDB), podemos adicionar uma seleção no campo `Major_Genre` para permitir a filtragem interativa por gênero de filme.

Para começar, vamos extrair os gêneros únicos (não nulos) dos dados de `movies (filmes)`:

In [ ]:
df = pd.read_json(movies) # carrega dados do filme
genres = df['Major_Genre'].unique() # obtém valores de campo exclusivos
genres = list(filter(lambda d: d is not None, genres)) # filtrar valores Nenhum
genres.sort() # classificar em ordem alfabética

Para uso posterior, vamos também definir uma lista de valores `MPAA_Rating` exclusivos:

In [ ]:
mpaa = ['G', 'PG', 'PG-13', 'R', 'NC-17', 'Not Rated']

Agora vamos criar uma seleção `single` (única) vinculada a um menu suspenso.

*Use o menu de consulta dinâmica abaixo para explorar os dados. Como as classificações variam por gênero? Como você revisaria o código para filtrar `MPAA_Rating` (G, PG, PG-13, etc.) em vez de `Major_Genre`?*

In [ ]:
selectGenre = alt.selection_single(
    name='Select', # nomeia a seleção para 'Select'
    fields=['Major_Genre'], # limita o campo de seleção para o 'Major_Genre'
    init={'Major_Genre': genres[0]}, # Utiliza a primeira entrada de gênero como valor inicial
    bind=alt.binding_select(options=genres) # faz um bind no menu para coletar nomes únicos de gêneros
)

alt.Chart(movies).mark_circle().add_selection(
    selectGenre
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(selectGenre, alt.value(0.75), alt.value(0.05))
)

Nossa construção acima aproveita vários aspectos das seleções:

- Damos um nome à seleção (`'Select'`). Este nome não é obrigatório, mas permite influenciar o texto do rótulo do menu de consulta dinâmica gerado. (O que acontece se você remover o nome? Experimente!)
- Restringimos a seleção a um campo de dados específico (`Major_Genre`). Anteriormente, quando usávamos uma seleção `single` (única), a seleção era mapeada para pontos de dados individuais. Ao limitar a seleção a um campo específico, podemos selecionar _all (todos)_ os pontos de dados cujo valor do campo `Major_Genre` corresponde ao único valor selecionado.
- Inicializamos `init=...` a seleção com um valor inicial.
- Nós `bind (vinculamos)` a seleção a um _widget_ de interface, neste caso um menu suspenso via `binding_select`.
- Como antes, usamos uma codificação condicional para controlar a opacidade do canal.





### 6.3.1. Vinculando seleções a múltiplas entradas

Uma instância de seleção pode ser vinculada a _múltiplos_ widgets de consulta dinâmica. Vamos modificar o exemplo acima para fornecer filtros para _ambos_ `Major_Genre` e `MPAA_Rating`, usando os famosos _"radio buttons"_ em vez de um menu. Nossa seleção `single (única)` agora é definida em um único _par_ de gênero e valores de classificação MPAA

Procure conjunções surpreendentes de gênero e classificação. Existe algum filme de terror com classificação G ou PG?





In [ ]:
# seleção de valor único em pares [Major_Genre, MPAA_Rating]
# usa valores específicos conectados como valores iniciais selecionados
selection = alt.selection_single(
    name='Select',
    fields=['Major_Genre', 'MPAA_Rating'],
    init={'Major_Genre': 'Drama', 'MPAA_Rating': 'R'},
    bind={'Major_Genre': alt.binding_select(options=genres), 'MPAA_Rating': alt.binding_radio(options=mpaa)}
)
  
# gráfico de dispersão, modifique a opacidade com base na seleção
alt.Chart(movies).mark_circle().add_selection(
    selection
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(selection, alt.value(0.75), alt.value(0.05))
)

_Curiosidades: a classificação PG-13 não existia quando os filmes [Jaws](https://www.imdb.com/title/tt0073195/) e [Jaws 2](https://www.imdb.com/title/tt0077766/) foram liberados. O primeiro filme a receber classificação PG-13 foi de 1984 [Red Dawn](https://www.imdb.com/title/tt0087985/)._

### 6.3.2. Usando visualizações como consultas dinâmicas

Embora os _widgets_ de interface padrão mostrem os possíveis valores dos parâmetros de consulta, eles não visualizam a distribuição desses valores. Também podemos querer usar interações mais ricas, como seleções de múltiplos valores ou intervalos, em vez de widgets de entrada que selecionam apenas um único valor por vez.

Para resolver esses problemas, podemos criar gráficos adicionais para visualizar dados e oferecer suporte a consultas dinâmicas. Vamos adicionar um histograma da contagem de filmes por ano e usar uma seleção de intervalo para destacar dinamicamente os filmes em períodos de tempo selecionados.

*Interaja com o histograma anual para explorar filmes de diferentes períodos. Você viu alguma evidência de [viés de amostragem](https://en.wikipedia.org/wiki/Sampling_bias) ao longo dos anos? (Como as avaliações do ano e dos críticos se relacionam?)*

Os anos vão de 1930 a 2040! Os filmes futuros estão em pré-produção ou existem erros de “um século”? Além disso, dependendo do fuso horário em que você está, você poderá ver um pequeno aumento em 1969 ou 1970. Por que isso aconteceria? (Veja o final do caderno para explicação!)




In [ ]:
brush = alt.selection_interval(
    encodings=['x'] # limita a seleção aos valores do eixo x (ano)
)

# histograma de consulta dinâmica
years = alt.Chart(movies).mark_bar().add_selection(
    brush
).encode(
    alt.X('year(Release_Date):T', title='Films by Release Year'),
    alt.Y('count():Q', title=None)
).properties(
    width=650,
    height=50
)

# gráfico de dispersão, modifique a opacidade com base na seleção
ratings = alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(brush, alt.value(0.75), alt.value(0.05))
).properties(
    width=650,
    height=400
)

alt.vconcat(years, ratings).properties(spacing=5)

O exemplo acima fornece consultas dinâmicas usando uma _seleção vinculada_ entre gráficos:

- Criamos uma seleção de `intervalo` (`brush`) e definimos `encodings=['x']` para limitar a seleção apenas ao eixo x, resultando em um intervalo de seleção unidimensional.
- Registramos `brush` com nosso histograma de filmes por ano via `.add_selection(brush)`.
- Usamos `brush` em uma codificação condicional para ajustar a `opacidade` do gráfico de dispersão.

Esta técnica de interação de selecionar elementos em um gráfico e ver destaques vinculados em um ou mais gráficos é conhecida como [_brushing &amp; linking_](https://en.wikipedia.org/wiki/Brushing_and_linking).



## 6.4 Arrastamento &amp; Zoom

O gráfico de dispersão de classificações de filmes está um pouco sobrecarregado em alguns lugares, tornando difícil examinar pontos em regiões mais densas. Usando as técnicas de interação de _panning_ (arrastamento) e _zoomimg_ (zoom), podemos inspecionar as regiões densas mais de perto.

Vamos começar pensando em como podemos expressar arrastamento e zoom usando seleções do _Altair_. O que define a "área visível" de um gráfico? Domínios de escala dos eixos!

Podemos alterar os domínios das escalas para modificar o intervalo de dados visualizados. Para fazer isso de maneira interativa, podemos vincular uma seleção de `interval` (intervalos) aos domínios das escalas com o código `bind='scales'`. O resultado é que, em vez de um intervalo de seleção que podemos arrastar e ampliar, podemos arrastar e ampliar toda a área do gráfico!

No gráfico abaixo, clique e arraste para deslocar (transladar) a visão ou use o _scroll_ para dar zoom (ampliar) a visão. O que você pode descobrir sobre a precisão dos valores de _rating_ fornecidos?

In [ ]:
alt.Chart(movies).mark_circle().add_selection(
    alt.selection_interval(bind='scales')
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # use minExtent para estabilizar a posição do título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
)

alt.Chart(...)

Ao dar zoom, podemos ver que os valores de _rating_ têm precisão limitada! Os _ratings_ do _Rotten Tomatoes_ são números inteiros, enquanto os do IMDB são truncadas para décimos. Como resultado, ocorre sobreposição de pontos mesmo quando damos zoom, com vários filmes compartilhando os mesmos valores de rating.

Ao ler o código acima, você pode notar o código `alt.Axis(minExtent=30)` no canal de codificação `y`. O parâmetro `minExtent` garante que um espaço mínimo seja reservado para as marcações e rótulos do eixo. Por que fazer isso? Quando fazemos o movimento de pan (arrastar) e zoom, os rótulos do eixo podem mudar e fazer com que a posição do título do eixo se desloque. Ao definir uma extensão mínima, podemos reduzir movimentos indesejados no gráfico. Experimente mudar o valor de `minExtent`, por exemplo, configurando-o para zero, e então dê zoom para fora para ver o que acontece quando rótulos de eixo mais longos entram na visualização.

O _Altair_ também inclui uma forma abreviada para adicionar arrastamento e zoom a um gráfico. Em vez de criar diretamente uma seleção, você pode chamar `.interactive()` para fazer o _Altair_ gerar automaticamente uma seleção de intervalo limitada aos eixos do gráfico:

In [ ]:
alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # use minExtent para estabilizar a posição do título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
).interactive()

alt.Chart(...)

Por padrão, as vinculações de escala para seleções incluem tanto os canais de codificação `x` quanto `y`. E se quisermos limitar o arrastamento e zoom a uma única dimensão? Podemos usar `encodings=['x']` para restringir a seleção ao canal `x` apenas:

In [ ]:
alt.Chart(movies).mark_circle().add_selection(
    alt.selection_interval(bind='scales', encodings=['x'])
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # use minExtent para estabilizar a posição do título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
)

alt.Chart(...)

Quando aplicamos zoom em apenas um dos eixos, o formato dos dados visualizados pode mudar, potencialmente afetando nossa percepção das relações nos dados. [Escolher uma proporção de aspecto adequada](http://vis.stanford.edu/papers/arclength-banking) é uma consideração importante no _design_ de visualizações!

## 6.5 Navegação: Visão Geral + Detalhe

Ao aplicar _panning_ (translação) e zoom, ajustamos diretamente a "janela de visualização" de um gráfico. A estratégia de navegação relacionada visão geral + detalhe utiliza, em vez disso, uma exibição geral para mostrar todos os dados, suportando seleções que arrastam e dão zoom em uma exibição de foco separada.

Abaixo, temos dois gráficos de área mostrando uma década de flutuações de preço do índice de ações S&amp;P 500. Inicialmente, ambos os gráficos exibem o mesmo intervalo de dados. Clique e arraste no gráfico inferior de visão geral para atualizar a exibição de foco e examinar períodos específicos.

In [15]:
brush = alt.selection_interval(encodings=['x']);

base = alt.Chart().mark_area().encode(
    alt.X('date:T', title=None),
    alt.Y('price:Q')
).properties(
    width=700
)
  
alt.vconcat(
    base.encode(alt.X('date:T', title=None, scale=alt.Scale(domain=brush))),
    base.add_selection(brush).properties(height=60),
    data=sp500
)

alt.VConcatChart(...)

Ao contrário do caso anterior de arrastar e dar zoom, aqui não queremos vincular uma seleção diretamente às escalas de um único gráfico interativo. Em vez disso, queremos vincular a seleção ao domínio da escala em outro gráfico. Para isso, atualizamos o canal de codificação `x` para o nosso gráfico de foco, definindo a propriedade `domain` da escala para referenciar nossa seleção `brush`. Se nenhum intervalo for definido (ou seja, se a seleção estiver vazia), o Altair ignora o _brush_ e usa os dados subjacentes para determinar o domínio. Quando um intervalo de _brush_ (intervalo de seleção) é criado, o _Altair_, contrariamente, o utiliza como o `domain` da escala para o gráfico de foco.

## 6.6 Detalhes sob Demanda

Uma vez que identificamos pontos de interesse dentro de uma visualização, frequentemente queremos saber mais sobre eles. _Detalhes sob demanda_ refere-se à indagação interativa por mais informações sobre os valores selecionados. _Tooltips_ são uma maneira útil de fornecer detalhes sob demanda. No entanto, os _tooltips_ tipicamente mostram informações para um ponto de dado por vez. Como podemos mostrar mais?

O gráfico de dispersão de ratings de filmes inclui vários _outliers_ (valores atípicos) potencialmente interessantes, onde as classificações do Rotten Tomatoes e do IMDB discordam. Vamos criar um gráfico que nos permita selecionar pontos interativamente e mostrar seus rótulos. Para acionar a consulta de filtro na interação de hover ou clique, usaremos o [operador de composição do Altair](https://altair-viz.github.io/user_guide/interactions.html#composing-multiple-selections) `|` ("ou").

Passe o _mouse_ sobre os pontos no gráfico de dispersão abaixo para ver um destaque e o título do rótulo. Clique com a tecla _Shift_ para fazer anotações persistentes e visualizar vários rótulos ao mesmo tempo. Quais filmes são adorados pelos críticos do _Rotten Tomatoes_, mas não pelo público geral no IMDB (ou vice-versa)? Veja se consegue encontrar possíveis erros, onde dois filmes diferentes com o mesmo nome foram acidentalmente combinados!

In [ ]:
hover = alt.selection_single(
    on='mouseover',  # selecionar ao passar o mouse
    nearest=True,    # selecionar o ponto mais próximo do cursor do mouse
    empty='none'     # seleção vazia não deve corresponder a nada
)

click = alt.selection_multi(
    empty='none' # seleção vazia não corresponde a pontos
)

# codificações do gráfico de dispersão compartilhadas por todas as marcas
plot = alt.Chart().mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q'
)
  
# base compartilhada para novas camadas
base = plot.transform_filter(
    hover | click # filtrar para pontos em qualquer seleção
)

# adicionar camadas de pontos do gráfico de dispersão, anotações de halo e rótulos de título
alt.layer(
    plot.add_selection(hover).add_selection(click),
    base.mark_point(size=100, stroke='firebrick', strokeWidth=1),
    base.mark_text(dx=4, dy=-8, align='right', stroke='white', strokeWidth=2).encode(text='Title:N'),
    base.mark_text(dx=4, dy=-8, align='right').encode(text='Title:N'),
    data=movies
).properties(
    width=600,
    height=450
)

alt.LayerChart(...)

O exemplo acima adiciona três novas camadas ao gráfico de dispersão: uma anotação circular, texto branco para fornecer um fundo legível e texto preto mostrando o título de um filme. Além disso, este exemplo usa duas seleções em conjunto:

1. Uma seleção simples (`hover`) que inclui `nearest=True` para selecionar automaticamente o ponto de dados mais próximo conforme o mouse se move.
2. Uma seleção múltipla (`click`) para criar seleções persistentes via _shift-click_.

Ambas as seleções incluem o conjunto `empty='none'` para indicar que nenhum ponto deve ser incluído se uma seleção estiver vazia. Essas seleções são então combinadas em um único predicado de filtro &mdash; o lógico _ou_ de `hover` e `click` &mdash; para incluir pontos que estejam em _qualquer uma_ das seleções. Usamos esse predicado a fim de filtrar as novas camadas para exibir anotações e rótulos apenas para os pontos selecionados.

Usando seleções e camadas, podemos realizar uma variedade de designs diferentes para detalhes sob demanda! Por exemplo, aqui está uma série temporal com escala logarítmica dos preços das ações de empresas de tecnologia, anotada com uma linha guia e rótulos para a data mais próxima ao cursor do mouse:

In [ ]:
# selecione um ponto para fornecer detalhes sob demanda
label = alt.selection_single(
    encodings=['x'], # limita a seleção ao valor do eixo x
    on='mouseover',  # seleciona nos eventos de mouseover (passar o mouse em cima)
    nearest=True,    # seleciona o ponto de dados mais próximo ao cursor
    empty='none'     # seleção vazia não inclui pontos de dados
)

# define nosso gráfico base de linha dos preços das ações
base = alt.Chart().mark_line().encode(
    alt.X('date:T'),
    alt.Y('price:Q', scale=alt.Scale(type='log')),
    alt.Color('symbol:N')
)

alt.layer(
    base, # gráfico base de linha
    
    # adiciona uma marca de linha vertical para servir como linha guia
    alt.Chart().mark_rule(color='#aaa').encode(
        x='date:T'
    ).transform_filter(label),
    
    # adiciona marcas de círculo para pontos de tempo selecionados, esconde pontos não selecionados
    base.mark_circle().encode(
        opacity=alt.condition(label, alt.value(1), alt.value(0))
    ).add_selection(label),

    # adiciona texto com contorno branco para fornecer um fundo legível para os rótulos
    base.mark_text(align='left', dx=5, dy=-5, stroke='white', strokeWidth=2).encode(
        text='price:Q'
    ).transform_filter(label),

    # adiciona rótulos de texto para os preços das ações
    base.mark_text(align='left', dx=5, dy=-5).encode(
        text='price:Q'
    ).transform_filter(label),
    
    data=stocks
).properties(
    width=700,
    height=400
)

alt.LayerChart(...)

Pondo em ação o que aprendemos até agora: você pode modificar o gráfico de dispersão dos filmes acima (o com a consulta dinâmica sobre os anos) para incluir uma marca `rule` que mostre o rating IMDB (ou _Rotten Tomatoes_) médio para os dados contidos na seleção de ano `interval`?

## Limpeza &amp; Conexão, Revisitado

Anteriormente neste _notebook_, vimos um exemplo de _limpeza &amp; conexão_: usando um histograma de consulta dinâmica para destacar pontos em um gráfico de dispersão de classificação de filme. Aqui, visitaremos alguns exemplos adicionais envolvendo seleções vinculadas.

Retornando ao dataset `cars`, podemos usar o operador `repeat` para construir uma [matriz de gráficos de dispersão (SPLOM)](https://en.wikipedia.org/wiki/Scatter_plot#Scatterplot_matrices) que mostra associações entre quilometragem, aceleração e potência. Podemos definir uma seleção de `interval` e incluí-la dentro de nossa especificação de gráfico de dispersão repetida para habilitar seleções vinculadas entre todos os gráficos.

Clique e arraste em qualquer um dos gráficos abaixo para executar a limpeza e conexão!

In [ ]:
brush = alt.selection_interval(
    resolve='global' # Resolve todas as seleções para uma única instância global
)

alt.Chart(cars).mark_circle().add_selection(
    brush
).encode(
    alt.X(alt.repeat('column'), type='quantitative'),
    alt.Y(alt.repeat('row'), type='quantitative'),
    color=alt.condition(brush, 'Cylinders:O', alt.value('grey')),
    opacity=alt.condition(brush, alt.value(0.8), alt.value(0.1))
).properties(
    width=140,
    height=140
).repeat(
    column=['Acceleration', 'Horsepower', 'Miles_per_Gallon'],
    row=['Miles_per_Gallon', 'Horsepower', 'Acceleration']
)

alt.RepeatChart(...)

Observe acima o uso de `resolve='global'` na seleção `interval`. A configuração padrão de `'global'` indica que em todos os gráficos apenas um pincel pode estar ativo por vez. No entanto, em alguns casos, podemos querer definir pincéis em vários gráficos e combinar os resultados. Se usarmos `'resolve='union'`, a seleção será a _união_ de todos os pincéis: se um ponto residir em qualquer pincel, ele será selecionado. Alternativamente, se usarmos `'resolve='intersect'`, a seleção consistirá na _interseção_ de todos os pincéis: apenas os pontos que residirem em todos os pincéis serão selecionados.

Tente definir o parâmetro `'resolve` como `'union'` e `'intersect'` e veja como isso altera a lógica de seleção resultante.

### Filtragem Cruzada

Os exemplos de limpeza &amp; conexão que vimos usam codificações condicionais, por exemplo, para alterar valores de opacidade em resposta a uma seleção. Outra opção é usar uma seleção definida em uma visualização para filtrar o conteúdo de outra visualização.

Vamos construir uma coleção de histogramas para o _dataset_ `flights`: `delay` (atraso, ou seja, quão cedo ou tarde um voo chega, em minutos), `distance` (distância do voo em milhas) e `time` (hora da partida). Usaremos o operador `repeat` para criar os histogramas e adicionaremos uma seleção de `interval` para o eixo `x` com pinceis resolvidos por meio de interseção.

Em particular, cada histograma consistirá em duas camadas: uma camada de fundo cinza e uma camada de primeiro plano azul, com a camada de primeiro plano filtrada por nossa interseção de seleções de pincéis. O resultado é uma interação de filtragem cruzada nos três gráficos!

Arraste os intervalos de pincel nos gráficos abaixo. Conforme você seleciona voos com atrasos de chegada mais longos ou mais curtos, como as distribuições de distância e tempo respondem?

In [19]:
brush = alt.selection_interval(
    encodings=['x'],
    resolve='intersect'
);

hist = alt.Chart().mark_bar().encode(
    alt.X(alt.repeat('row'), type='quantitative',
        bin=alt.Bin(maxbins=100, minstep=1), # up to 100 bins
        axis=alt.Axis(format='d', titleAnchor='start') # integer format, left-aligned title
    ),
    alt.Y('count():Q', title=None) # no y-axis title
)
  
alt.layer(
    hist.add_selection(brush).encode(color=alt.value('lightgrey')),
    hist.transform_filter(brush)
).properties(
    width=900,
    height=100
).repeat(
    row=['delay', 'distance', 'time'],
    data=flights
).transform_calculate(
    delay='datum.delay < 180 ? datum.delay : 180', # clamp delays > 3 hours
    time='hours(datum.date) + minutes(datum.date) / 60' # fractional hours
).configure_view(
    stroke='transparent' # no outline
)

alt.RepeatChart(...)

_Ao fazer a filtragem cruzada, você pode observar que voos atrasados ​​têm mais probabilidade de partir em horários mais tardios. Esse fenômeno é familiar para passageiros frequentes: um atraso pode se propagar ao longo do dia, afetando viagens subsequentes daquele avião. Por isso, para melhores chances de uma chegada pontual, reserve um voo antecipado!_

A combinação de múltiplas visualizações e seleções interativas pode permitir formas valiosas de raciocínio multidimensional, transformando até mesmo histogramas básicos em poderosos dispositivos de entrada para fazer perguntas sobre um dataset!

## 6.8. Resumo

Para mais informações sobre as opções de interação suportadas no _Altair_, consulte o [Altair interactive selection documentation](https://altair-viz.github.io/user_guide/interactions.html). Para detalhes sobre como personalizar manipuladores de eventos, por exemplo, para combinar várias técnicas de interação ou suportar entrada baseada em toque em dispositivos móveis, consulte o [Vega-Lite selection documentation](https://vega.github.io/vega-lite/docs/selection.html).

Interessado em aprender mais?


*   A abstração de seleção foi introduzida no artigo [Vega-Lite: A Grammar of Interactive Graphics](http://idl.cs.washington.edu/papers/vega-lite/), de Satyanarayan, Moritz, Wongsuphasawat e Heer.
*   O sistema PRIM-9 (para projeção, rotação, isolamento e mascaramento em até 9 dimensões) é uma das primeiras ferramentas de visualização interativa, criada no início dos anos 1970 por Fisherkeller, Tukey e Friedman. [A retro demo video survives!](https://www.youtube.com/watch?v=B7XoW2qiFUA)
*   O conceito de brushing e linking foi cristalizado por Becker, Cleveland e Wilks em seu artigo de 1987  [Dynamic Graphics for Data Analysis](https://scholar.google.com/scholar?cluster=14817303117298653693).
*   Para um resumo abrangente das técnicas de interação para visualização, consulte [Interactive Dynamics for Visual Analysis](https://queue.acm.org/detail.cfm?id=2146416) , de Heer e Shneiderman.
*   Finalmente, para um tratado sobre o que torna a interação eficaz, leia o clássico artigo  [Direct Manipulation Interfaces](https://scholar.google.com/scholar?cluster=15702972136892195211), de Hutchins, Hollan e Norman.






## 6.8.1. Apêndice: Sobre a Representação do Tempo

Anteriormente, observamos um pequeno aumento no número de filmes em 1969 e 1970. De onde vem esse aumento? E por que 1969 ou 1970? A resposta vem de uma combinação de dados ausentes e de como o seu computador representa o tempo.

Internamente, datas e horários são representados em relação à [**época UNIX**](https://en.wikipedia.org/wiki/Unix_time), na qual o “zero” do tempo corresponde à meia-noite de 1º de janeiro de 1970 no [**horário UTC**](https://en.wikipedia.org/wiki/Coordinated_Universal_Time), que segue o[ **meridiano de Greenwich**](https://en.wikipedia.org/wiki/Prime_meridian). Acontece que há alguns filmes com datas de lançamento ausentes (`null`). Esses valores `null` são interpretados como tempo `0`, e, portanto, mapeados para 1º de janeiro de 1970 no horário UTC. Se você vive nas Américas – e, portanto, em fusos horários “anteriores” – esse ponto preciso no tempo corresponde a uma hora mais cedo em 31 de dezembro de 1969 no seu fuso horário local. Por outro lado, se você mora perto ou a leste do meridiano de Greenwich, a data no seu fuso horário local será 1º de janeiro de 1970.

A lição? Sempre seja cético em relação aos seus dados e tenha em mente que a forma como os dados são representados (seja como datas e horários, números de ponto flutuante, latitudes e longitudes, etc.) pode, às vezes, gerar artefatos que impactam a análise!